# Web Scraping Teil 1

In diesem zweiteiligen Notebook lernen wir, wie wir mit **wenig Code** sehr **viel Inhalt** aus dem Internet herunterladen und für die Analyse aufbereiten können. Die dazu notwendige Technik nennt sich, wie schon mehrfach erwähnt, *Web Scraping*. Das englische Verb *to scrape* bedeutet "schürfen" – wir schürfen bloß nicht Gold, sondern (potenziell goldwerte) Inhalte aus dem *Web*. ✨

Zu Beginn des Notebooks lernst Du wichtige Hintergründe zum Web Scraping. Danach wird es deutlich angewandter. Einige Inhalte sind eingeklappt, da sie nicht zwingend notwendig sind, um "mitzukommen". Lies sie gerne für ein vertieftes Verständnis des Themas.

<details>
    <summary><b>❓ Warum lohnt sich Web Scraping?</b></summary>
    <br>Das Notebook "Einführung" hielt gleich zu Beginn fest, dass Web Scraping mit ein Grund ist, warum sich Programmierkenntnisse lohnen. Als Beispiel diente ein Forschungsprojekt, in dem Bildunterschriften in Artikeln bei ZEIT Online gesammelt werden sollten. Während die manuelle Herangehensweise (Aufruf jedes Artikels im Browser, Markieren und Kopieren der Bildunterschriften, Einfügen in eine Excel-Tabelle, Anreichern mit Metadaten wie Artikellink und -titel) bei einem quantitativen Vorhaben vollkommen impraktikabel ist, können wenige Zeilen Web Scraping-Code diese Aufgabe im Fluge erledigen.<br><br>Web Scraping eröffnet Geistes- und Sozialwissenschaftler:innen neue Möglichkeiten, indem es die stetig wachsenden Mengen an Daten aus dem Internet für die Forschung nutzbar macht. Von online verfügbaren Werken (z.&nbsp;B. beim <a href="https://www.projekt-gutenberg.org/info/texte/index.html">Projekt Gutenberg</a>, s.&nbsp;u.), über Kommentare unter Zeitungsartikeln bis hin zu Restaurantbewertungen bei Tripadvisor lassen sich die meisten Webinhalte so effizient und flexibel herunterladen. Web Scraping-Skills schließen damit optimal an unsere bereits erlernten Fertigkeiten an, Daten aufzubereiten sowie zu analysieren (Notebook "Datenanalyse"), machen sie in vielen Fällen die notwendigen Daten doch erst überhaupt verfügbar.<br><br>
</details>

<details>
<summary><b>❗️ Abgrenzung zu APIs</b></summary>
    <br>Manche Plattformen (etwa soziale Medien, Behörden oder Datenbanken) bieten auch sog. <i>APIs</i> (auf Deutsch: <i>Programmierschnittstellen</i>), über die (ausgewählte) Daten abgefragt werden können. In den Zusatzübungen zum Notebook "Reguläre Ausdrücke" benutzen wir etwa die offizielle API von Wikipedia. APIs sind jedoch sehr plattformspezifisch zu bedienen, unterliegen häufigen Änderungen und werden bisweilen ganz deaktiviert. Außerdem erfolgt ihr Zugriff meist direkt über die Command Line, es sei denn es gibt einen sog. <i>Python Wrapper</i> (wie für die Wikipedia-API). Aus diesen Gründen behandelt dieses Notebook nur die etwas rohere, aber auch wesentlich flexiblere Technik des Web Scraping.
</details>

Web Scraping besteht grundsätzlich aus **zwei Schritten**:

1. **Abruf**: Wir rufen iterativ eine Webseite nach der anderen auf und speichern den ihnen zugrundeliegenden, sog. *Quelltext* (s.&nbsp;u.) ab.
2. **Extraktion**: Da wir i.&nbsp;d.&nbsp;R. nur an einem bestimmten Teil des Quelltexts interessiert sind (z.&nbsp;B. an den erwähnten Bildunterschriften oder an den Pseudonymen von Kommentarschreibenden), extrahieren wir nun die relevanten Informationen aus dem Quelltext, d.&nbsp;h. "wir trennen die Spreu vom Weizen".

Unten gehen wir auf beide Schritte detailliert ein. Zunächst wird der Anwendungsfall vorgestellt, den Du im zweiten Teil des Notebooks "Web Scraping" bearbeiten wirst. Anschließend werfen wir einen Blick hinter die Kulissen von Webseiten. Ein grundlegendes Verständnis vom Aufbau einer Webseite, also vom Quelltext, ist insbesondere für den Extraktionsschritt unerlässlich. 

***

## 🔧 Anwendungsfall: Den Faust I scrapen 🎭

*Faust I* von Johann Wolfgang von Goethe ist eines der bedeutendsten Werke der deutschsprachigen Literatur. Eine Vielzahl an (quantitativen) Fragestellungen ließe sich an die Tragödie herantragen, z.&nbsp;B. könnten wir die Reinheit der Reimmuster untersuchen. Dank unserer stets wachsenden Python-Kenntnisse wären wir dazu mehr und mehr in der Lage. 

Im Anwendungsfall in diesem Notebook soll es aber nicht um die Daten*auswertung* gehen, sondern um die vorgelagerte Daten*beschaffung*. Das gesamte Werk findet sich auf insgesamt 27 Seiten verteilt frei verfügbar bei [Projekt Gutenberg](https://www.projekt-gutenberg.org/goethe/faust1/chap002.html). Wir wollen es automatisiert herunterladen. Konkret wird es Deine Aufgabe sein, sämtliche Strophen des Dramas inkl. der Figuren, die sie sprechen, in eine Textdatei zu überführen. Regieanweisungen und Szenentitel ignorieren wir dabei. 

Da es sich "nur" um 27 Seiten handelt, würde es wohl nicht allzu lange dauern, den gesamten Werkinhalt auf jeder Seite zu markieren, zu kopieren und in eine Textdatei einzufügen. Blöderweise befinden sich jedoch zwischen den einzelnen Strophen und manchmal sogar in den Versen versteckt jede Menge Regieanweisungen. Es käme einer Sisyphusarbeit gleich, diese zuverlässig von Hand auszusortieren. Glücklicherweise ist der Quelltext der Webseiten aber so strukturiert, dass wir beim Web Scraping zielgerichtet nur die relevanten Textteile extrahieren können. Deswegen sowie aufgrund der Tatsache, dass wir unseren fertigen Code mit wenigen Anpassungen für die meisten der über 10.000 Werke von Projekt Gutenberg verwenden können, ist das automatisierte Web Scraping der manuellen Herangehensweise klar überlegen. 

Die Textdatei soll final wie folgt aussehen (der Auszug entstammt der Szene [*Nacht*](https://www.projekt-gutenberg.org/goethe/faust1/chap004.html)):

    Geist

        Wer ruft mir?

    Faust

        Schreckliches Gesicht!

    Geist

        Du hast mich mächtig angezogen,
        An meiner Sphäre lang gesogen,
        Und nun –

    Faust

        Weh! ich ertrag dich nicht!
        
Wie gewohnt kannst Du den Anwendungsfall später ohne weitere Anleitung in Angriff nehmen oder die Schritt-für-Schritt-Anweisungen befolgen. 

***

## Hinter den Kulissen von Webseiten

Alle Informationen, die Dein Browser braucht, um die Inhalte einer Webseite korrekt anzuzeigen (bzw. zu *rendern*), sind im Quelltext der Webseite enthalten. Bevor wir lernen, Quelltexte automatisiert "anzuzapfen", wollen wir den Schritt erst manuell durchführen. Wie auch im Notebook "Reguläre Ausdrücke" beschrieben, können wir uns den Quelltext jeder beliebigen Webseite wie folgt im Browser anzeigen lassen:

- bei Google Chrome und Firefox mittels Rechtsklick "Seitenquelltext anzeigen" wählen
- bei Safari mittels Rechtsklick "Seitenquelltext einblenden" wählen

Der gesamte Quelltext ist i.&nbsp;d.&nbsp;R. sehr umfassend und meist sind wir wie erwähnt nur an bestimmten Elementen darin interessiert. Um ein solches Element innerhalb des Quelltexts zu lokalisieren, können wir es im Browser markieren und anschließend

- bei Google Chrome und Firefox mittels Rechtsklick "Untersuchen" wählen
- bei Safari mittels Rechtsklick "Element-Informationen" wählen

So können wir uns etwa anzeigen lassen, wo im Quelltext die Überschrift der Webseite des [Projekt Gutenberg](https://www.projekt-gutenberg.org) definiert ist ("Startseite", vgl. rote/runde Markierung; Stand: 03/24), nämlich einigermaßen verschachtelt in einem Element mit dem sog. *Tag* `<h3>`, wie der rechte Ausschnitt des folgenden Screenshots zeigt. Zu Elementen, Tags etc. gleich mehr!

<img src="../3_Dateien/Grafiken_und_Videos/Seitenquelltext.png">

Um ein Gefühl für das "Gerüst" einer Webseite zu erhalten, üben wir diesen manuellen Abruf von Quelltexten. Analog zur Übung 3 im Notebook "Reguläre Ausdrücke" führen wir die erste Übung nicht mithilfe von Python durch, sondern verwenden [Sublime Text](https://www.sublimetext.com) (vgl. Notebook "Funktionen und Methoden Teil 2" sowie "Reguläre Ausdrücke"), das insbesondere unter Verwendung von Tastenkombinationen äußerst nützlich ist. In der Übung verwenden wir reguläre Ausdrücke. Solltest Du damit noch nicht vertraut sein, extrahier stattdessen einen anderen, möglichst oft vorkommenden literalen Suchbegriff aus dem Quelltext.

***

✏️ **Übung 1:** Besuch eine beliebige Webseite in einem Browser Deiner Wahl. Die Webseite sollte mehrere Links zu anderen Seiten beinhalten, was auf die meisten Webseiten zutrifft. Lass Dir wie oben beschrieben den Quelltext der Seite anzeigen. Geh nun wie folgt vor und verwend dabei die angegebenen Tastenkombinationen.

1. Markier den gesamten Quelltext (`Strg` + `a` (Windows und Linux) bzw. `⌘` + `a` (macOS)), kopier ihn (`Strg`/`⌘` + `c`) und füg ihn in ein leeres Dokument bei Sublime Text ein (`Strg`/`⌘` + `v`). 
2. Formulier einen regulären Ausdruck, der sämtliche Links in Deinem Quelltext matcht. Manche Webseiten enthalten komplette Links in ihrem Quelltext (z.&nbsp;B. "ht<span>tps://www.</span>domain<span>.de/</span>subdomain/page"), andere wiederum bloß abgekürzte Links ("subdomain/page"), die, wenn man im Browser draufklickt, um den gemeinsamen "Stammlink" ergänzt werden. Dein regulärer Ausdruck soll sich nach dem Linkformat im Quelltext Deiner Webseite richten und möglichst viele Links matchen.
3. Aktivier die Suche bei Sublime Text über `Strg`/`⌘` + `f`). Gib den regulären Ausdruck ein und such bzw. markier alle matches über die Tastenkombination `Alt`/`option` + `Enter` (dies entspricht der Schaltfläche "Find All"). Ggf. musst Du reguläre Ausdrücke für die Suche aktivieren (links neben dem Suchfeld über `.*`).
5. Kopier alle Links. Öffne ein weiteres Dokument bei Sublime Text und füg die Links dort ein. Führ auch diese Schritte nach Möglichkeit mithilfe von Tastenkombinationen aus.

<details>
  <summary>💡 Tipp </summary>
  <br>Folgender Screenshot zeigt links einen Ausschnitt des Quelltexts des <a href="https://de.wiktionary.org/wiki/Internet" title="Wiktionary-Eintrag zu 'Internet'">Wiktionary-Eintrags zu "Internet"</a> inklusive markierter matches des folgenden regulären Ausdrucks, der sich zwar nicht als perfekt, aber als ausreichend praktikabel erwies:<br><br><code>"(https:)?//?\S+"</code><br><br>Rechts zeigt der Screenshot die in ein neues Dokument eingefügten Links, bei denen noch die Anführungszeichen weggestript werden müssten. Ebenso müssten die abgekürzten Links um ihren Stammlink ergänzt werden, damit sie aufgerufen werden können.<br><br><img src="../3_Dateien/Grafiken_und_Videos/uebung1_web_scraping.png">
</details>

***

## HTML

Der Quelltext einer Webseite ist üblicherweise in der sog. *Auszeichnungssprache* HTML (ausgeschrieben: *<u>H</u>yper<u>t</u>ext <u>M</u>arkup <u>L</u>anguage*) verfasst – eine maschinenlesbare Sprache, die genaue Anweisungen liefert, was wo wie dargestellt werden soll. Ohne HTML (oder eine andere Auszeichnungssprache) würden sämtliche Inhalte einer Webseite unstrukturiert nebeneinander angezeigt werden (vgl. rechts im Screenshot unten). HTML sorgt dafür, dass die Inhalte sinnvoll in *Header*, Überschriften, ein oder mehrere Spalten in sinnvollen Abmessungen, *Footer* etc. angezeigt werden (links im Screenshot). Die gezeigte fiktive Webseite ist zwar äußerst grob strukturiert, dennoch wird klar, dass die Strukturierung von Inhalten ein Muss ist. 

<img src="../3_Dateien/Grafiken_und_Videos/strukturiert_vs_unstrukturiert.png">

<details>
    <summary><b>❗️ Statische vs. dynamische Webseiten</b></summary>
    <br>Traditionellerweise sind alle Webseiten <b>statisch</b> aufgebaut. Der Quelltext einer statischen Webseite liegt fertig gecodet auf dem Server. Wenn eine Benutzerin eine solche Webseite aufruft, muss der fertige Quelltext nur noch an ihren Browser übermittelt werden. Unabhängig davon, wer die statische Webseite aufruft, sie sieht für alle gleich aus.
    <br><br>Moderne Webseiten sind allerdings meistens <b>dynamisch</b> aufgebaut, d.&nbsp;h. ihr Quelltext wird (teilweise) erst beim Aufruf der Seite generiert, abhängig u.&nbsp;a. vom Standort der Benutzerin und ihren vorherigen Browseraktivitäten. Je nach Benutzer:in wird eine etwas andere Webseite an den Browser übermittelt. Dynamische Webseiten beinhalten häufig auch Elemente, mit denen die Benutzerin interagieren kann, ohne dass die Webseite komplett neugeladen wird, etwa ausklappbare Textfelder oder Buttons zum Laden von weiteren Inhalten (z.&nbsp;B. Kommentare). 
    <br><br>Beim Web Scraping können wir grundsätzlich beide Arten von Webseiten herunterladen, denn auch bei dynamischen Webseiten wird schlussendlich ein HTML-Quelltext zur Übermittlung generiert. Dynamische Webseiten stellen uns als Web Scrapende aber vor zwei Herausforderungen: Erstens benutzen wir statt eines Browsers das Python-Modul <code>requests</code> (s.&nbsp;u.), um Quelltexte herunterzuladen. Personalisierte Inhalte, etwa basierend auf unserem Loginstatus, können wir so nicht erhalten. Zweitens enthält der gescrapte Quelltext nur, was der Browser auch direkt nach dem Laden der Webseite anzeigen würde. Standardmäßig eingeklappte Elemente (etwa der zweite Teil bei langen Kommentaren, der erst durch Klicken auf einen "Mehr"-Button nachgeladen würde) erhalten wir so nicht. Um diesen Einschränkungen zu entgehen, bietet sich das Modul <code><a href="https://selenium-python.readthedocs.io/installation.html">Selenium</a></code> an, das einen Browser emuliert. In diesem Notebook beschränken wir uns der Einfachheit halber auf Quelltexte ohne individualisierte bzw. nachgeladene Inhalte, wie sie uns <code>requests</code> ohne Tamtam liefert.
</details>

Nun ist es nicht unser Ziel, selbst (gute) Webseiten zu coden, sondern interessante Inhalte von bestehenden Webseiten "bloß" zu extrahieren. Die folgenden Erläuterungen zu HTML umfassen also nur das, was fürs Web Scraping relevant ist. 

### Grundstruktur

HTML strukturiert sämtliche Inhalte von Webseiten in Form von **Elementen** hierarchisch. Wie bei einem Stammbaum steht jedes Element in einer Abhängigkeitsbeziehung zum ihm übergeordneten Element und ggf. zu einem oder mehreren ihm untergeordneten Elementen. 

Drei Aspekte charakterisieren jedes Element:

- **Tag**: Jedes Element hat ein Tag, das mehr oder weniger eindeutige Strukturinformationen vorgibt. Das Tag `<h1>` für Überschriften etwa spezifiziert eine große, fett geschriebene Schrift. Genauer gesagt gibt es jeweils ein Starttag am Anfang jedes Elements (`<tag>`) und ein Endtag (`</tag>`) an dessen Ende (Ausnahme: inhaltsleere Elemente wie `<img>`, die kein Endtag haben, s.&nbsp;u.). Tags sind bei HTML vorgegeben – eine Übersicht über die wichtigsten aus Web Scraping-Perspektive folgt unten.
- **Attribute**: Neben dem Tag kann ein Element optional Attribute umfassen, die weitere Eigenschaften des Elements in Form von Schlüssel-Werte-Paaren definieren. Ein Element kann dadurch etwa einer bestimmten Klasse von Elementen zugeordnet werden, die wiederum auf eine bestimmte Art formatiert wird. 
- **Inhalt**: Meistens enthält das Element Inhalt in Form von Text oder in Form untergeordneter (verschachtelter) Elemente. Textinhalt wird nicht von Anführungszeichen umrahmt.

Daraus folgt diese **grundlegende HTML-Syntax** für ein Element mit keinem, einem oder mehreren Attributen sowie Textinhalt:

- `<tag>content</tag>` (keine Attribute)
- `<tag attribute="value">content</tag>` (ein Attribut)
- `<tag attribute1="value1" attribute2="value2">content</tag>` (zwei Attribute)

Wie einem Element andere Elemente untergeordnet werden (also verschachtelter Inhalt), sehen wir gleich. 

Jeder HTML-Code beginnt mit `<!DOCTYPE html>`, was den Dokumenttyp deklariert. Darauf folgt das oberste Element in der Hierarchie, der *Stamm* bzw. die *Wurzel* (engl. *root*), nämlich ein Element mit dem Tag `<html>`. Dieses Element hat wiederum immer zwei *Kinder*: `<head>` und `<body>`. In `<head>` befinden sich für uns i.&nbsp;d.&nbsp;R. uninteressante Daten wie der Titel der Webseite (der im Browsertab angezeigt wird) oder die verwendete Zeichencodierung. Der komplette Inhalt der Webseite befindet sich in `<body>`. Diese Grundstruktur zeigt sich sowohl im Screenshot des Quelltexts von www.projekt-gutenberg.org oben (das `<head>`-Element ist eingeklappt) als auch in der nächste Zelle. 

Das Element `<body>` umfasst hier eine große Überschrift (`<h1>`), einen normalen Textabschnitt (`<p>`), eine kleinere Überschrift (`<h2>`), gefolgt von noch einem Textabschnitt (`<p>`) sowie einer Liste (`<ul>`) mit drei Unterpunkten (`<li>`). Das Listenelement mit seinen Unterpunkten exemplifiziert verschachtelten Inhalt (achte darauf, wo sich das Endtag des `<ul>`-Elements befindet).

Wenn Du die Zelle obendran vom Typ "Raw" auf "Markdown" änderst und die Zelle ausführst, wird der HTML-Code gerendert, wie es ein Browser auch tun würde, wenn er den entsprechenden Quelltext darstellen müsste (abgesehen vom Titel der Webseite, den der Browser im Tab anzeigen würde). Das liegt daran, dass Markdown ebenfalls eine Auszeichnungssprache ist, die ihrerseits mit HTML-Code umgehen kann (vgl. auch die mittels HTML erzeugte zweispaltige Formatierung bei Übung 2 unten – indem Du in die Zelle doppelklickst, kannst Du den ungerenderten HTML-Code sehen).

***

<!-- Da es sich um in Markdown eingebetteten HTML-Code handelt, ist die grundlegende HTML-Struktur mit Dokumenttypdeklaration, html- und body-Element nicht nötig. Markdown-Code kommt ohne diese Struktur aus. Dieser Text hier ist übrigens ein Kommentar, der nicht gerendert wird.-->

<table style="font-size: 16px; letter-spacing: 0.2px;">
      <tr>
        <td>
            <p>✏️ <b>Übung 2:</b> Um die Struktur von HTML-Code weiter zu verinnerlichen, wollen wir trotzdem eine (sehr basale) Webseite schreiben. 
                Deine Aufgabe ist es, die im Screenshot gezeigte Rezeptwebseite nachzuprogrammieren. 
                Viel mehr als die oben gegebene HTML-Grundstruktur benötigst Du dafür nicht. Bloß zwei Dinge fehlen Dir:</p>
            <ol>
                <li>Um den Link zum <a href="https://www.eat-this.org/baba-ganoush-orientalischer-auberginensalat/#recipe">Originalrezept</a> einzubetten, 
                    kannst Du folgende Syntax verwenden, wobei Du den Link bei "link" innerhalb der Anführungszeichen einsetzt.<br><br>
                    <code>&lt;a href="link"&gt;Eat this&lt;/a&gt;</code></li><br>
                <li>Um eine Liste zu nummerieren, verwendest Du das Tag <code>&lt;ol&gt;</code> für <u>o</u>rdered <u>l</u>ist 
                    statt <code>&lt;ul&gt;</code> (<u>u</u>nordered <u>l</u>ist).</li>
            </ol>
            <p>Schreib den Code in einem Sublime Text-Dokument und speichere es anschließend mit der Endung ".html". 
               Wenn Du es nun in einem beliebigen Browser öffnest, sollte Dir Deine Rezeptwebseite angezeigt werden.</p>
        </td>
        <td>
            <img src="../3_Dateien/Grafiken_und_Videos/lieblingsrezept.png" width="1000">
        </td> 
      </tr>
</table>

***

### Tags

Nun schauen wir uns HTML-Tags an, die typischerweise interessante Inhalte umfassen. Wir unterscheiden grob zwischen strukturierenden Elementen (sog. *Blockelemente*) und formatierenden Elementen (sog. *Inline-Elemente*). Erstere grenzen Inhalte voneinander ab, z.&nbsp;B. mithilfe von Text*blöcken* oder Spalten. Sie beginnen standardmäßig auf einer neuen Zeile. Letzere nehmen den Feinschliff vor, zumeist innerhalb von Blockelementen, z.&nbsp;B. indem sie Text **fett** oder *kursiv* setzen. 

Vorerst reicht es, wenn Du die folgende Zusammenstellung überfliegst, um einen Eindruck der wichtigsten Tags zu erhalten. Später kannst Du hier nachschlagen.

| Tag | Verwendung |
|-|-|
|
|**Strukturierende Elemente**
|`<div>` | generisches Blockelement, um Inhalte zu strukturieren |
|`<main>`, `<article>`, `<section>`| weitere Blockelemente, um Inhalte zu strukturieren | 
|`<header>`, `<footer>`| Blockelemente für einführenden bzw. abschließenden Inhalt (etwa Veröffentlichungsdatum bzw. Copyright)| 
|`<h1>` bis `<h6>`| Blockelemente für große bis kleine Überschriften |
|`<p>`| Blockelement für Text | 
|`<table>`, `<th>`, `<tr>`, `<td>`| generiert eine (üblicherweise unsichtbare) Tabelle zur Anordnung von Inhalten in mehreren Spalten (`<table>`); `<th>` enthält Spaltenüberschriften, `<tr>` Zeileninhalte und `<td>` Spalteninhalte, vgl. Übung 2 oben | 
|`<ul>`, `<ol>`, `<li>`| generiert Liste mit Punkten (`<ul>`) oder Nummerierung (`<ol>`), wobei einzelne Punkte in `<li>` enthalten sind | 
|`<img>`, `<audio>`, `<video>` | Blockelemente für Bilder, Audio- und Videoinhalte; Quelle steht im `src`-Attribut: `<img src="path">`; kein Elementinhalt (Endtag entfällt)
|
| **Formatierende Elemente**
|`<span>`| generisches Inline-Element für Text; mithilfe von Attributen wird darin enthaltener Text formatiert |
|`<a>`| Inline-Element für klickbaren Link (Link steht im `href`-Attribut, angezeigter Text im Elementinhalt: `<a href="https://leo.org">Wörterbuch</a>`) | 
|`<b>`, `<i>`, `<u>`, `<s>`, `<code>` | Inline-Elemente, um Text **fett** (`<b>`), *kursiv* (`<i>`), <u>unterstrichen</u> (`<u>`), <s>durchgestrichen</s> (`<s>`) oder als `code` (`<code>`) zu formatieren | 

### Attribute

Wie erwähnt können Tags zusätzlich Attribute besitzen. Im Web Scraping-Zusammenhang ist besonders das `href`-Attribut bei Elementen mit `<a>`-Tag interessant, etwa wenn wir basierend auf dem Quelltext einer "Startseite" alle Links zu Unterseiten ausfindig machen wollen, um diese anschließend ebenfalls zu scrapen. 

Weiter ist das Attribut `class` spannend, das von Webseitenerstellenden bei verschiedenen Tags dazu genutzt wird, ähnliche Inhalte gleich zu formatieren. Für jede Klasse von Elementen kann nämlich ein bestimmter "Style" definiert werden. Nun interessiert uns nicht die Formatierung per se, wohl aber können wir uns die Gruppierung zunutze machen, um ähnliche Elemente zielgerichtet zu extrahieren. 

**Beim Extraktionsschritt kombinieren wir i.&nbsp;d.&nbsp;R. Tags und Attribute, um in zumeist endlosen Quelltexten genau das "herauszufischen", was uns interessiert.**

Schließen wir den theoretischen Teil mit zwei Übungen ab.

***

✏️ **Übung 3:** In folgenden HTML-Code haben sich diverse Fehler eingeschlichen. Wie viele findest Du? Beheb sie alle.  

***

✏️ **Übung 4:** Reproduzier folgenden Text mithilfe von HTML. Achte auf die korrekte Hierarchie der Elemente sowie darauf, alle Tags zu schließen. Handelt es sich um formatierende Inline-Elemente oder um strukturierende Blockelemente?

<img src="../3_Dateien/Grafiken_und_Videos/HTML_Formatierung.png">

***

Nun folgt der angewandte Teil. 

Zunächst schauen wir uns die Möglichkeit an, ganz bestimmte Elemente von Webseiten zu scrapen (z.&nbsp;B. Überschriften oder Links). Dazu müssen wir uns im zweiten Schritt, der Extraktion, vertieft mit dem HTML-Quelltext auseinandersetzen. Das ist mitunter aufwendig, andererseits erlaubt es uns, Inhalte zielgerichtet "anzusprechen".

Zum Abschluss des ersten Teils dieses Notebooks lernen wir dann noch eine simplere Option kennen, um ganz einfach den "Haupttext" einer Webseite zu scrapen (bei einer Nachrichtenseite etwa der eigentliche Artikeltext). Dabei fällt die Auseinandersetzung mit dem Quelltext weg, gleichzeitig können wir nicht mehr punktgenau steuern, welche Inhalte wir sammeln. Je nach Anwendungsfall eignet sich eine der beiden Herangehensweisen besser.

## Abruf

Für den ersten Schritt, das Abrufen von Quelltexten, installieren wir zunächst das dafür verwendete Modul `requests` über die Command Line: `pip3 install requests`. Anschließend importieren wir es:

In [1]:
import requests

Den Quelltext einer Webseite abzurufen ist denkbar einfach. Wir müssen bloß der `get`-Funktion des Moduls den gewünschten URL, also den Link, übergeben, hier etwa zum Wikipedia-Artikel unserer Lieblingsprogrammiersprache:

In [2]:
link = "https://de.wikipedia.org/wiki/Python_(Programmiersprache)"
response = requests.get(link)

⚠️ Achtung: Führ diesen Code am besten nur einmal aus. Dies gilt auch für alle weiteren Codes hier und in Deinen eigenen Skripten, in denen Inhalte von einem Server heruntergeladen werden. Bei häufiger Ausführung innerhalb kurzer Zeit kann es sein, dass Du vom Server **blockiert** wirst. Mehr zu verantwortungsvollem Web Scraping findest Du im zweiten Teil des Notebooks.

`source_code` enthält jetzt den Quelltext der Wikipedia-Seite zu unserer Programmiersprache. Es handelt sich aber nicht um einen gewöhnlichen string, sondern um ein sog. `Response`-Objekt. Wenn wir uns das Objekt einfach ausgeben lassen, erhalten wir den HTTP-Statuscode vom Abruf der Webseite:

In [3]:
response

<Response [200]>

200 und alle anderen Codes beginnend mit einer Zwei bedeuten, dass alles geklappt hat. Andere Codes wie 404 bedeuten, dass der Quelltext nicht abgerufen werden konnte. [Hier](https://de.wikipedia.org/wiki/HTTP-Statuscode#Liste_der_HTTP-Statuscodes) erfährst Du mehr über die verschiedenen Statuscodes.

Indem wir das Attribut `encoding` anhängen, können wir in Erfahrung bringen, wie die Webseite enkodiert ist (vgl. Notebooks "Input und Output Teil 1").

In [4]:
response.encoding #Nach Attributen stehen keine Klammern.

'UTF-8'

⚠️ Achtung: Verwirrenderweise bezieht sich der Begriff *Attribut* auf zwei verschiedene Dinge in diesem Notebook: Einerseits sind damit bestimmte "Teile" eines Objekts gemeint (hier das Encoding oder der eigentliche Quelltext des `Response`-Objekts, weiter unten auch der Textinhalt eines `BeautifulSoup`-Objekts), andererseits die Schlüssel-Werte-Paare, die optional Eigenschaften von HTML-Elementen definieren (s.&nbsp;o.).

Um das Encoding herauszufinden, schaut `requests` im `<head>`-Element des gescrapten Quelltexts nach (s.&nbsp;o.). Falls diese Angabe dort fehlt oder falsch ist, könnten wir diesem Attribut auch einfach den korrekten Wert zuweisen.

Auf den *eigentlichen* Quelltext greifen wir über das Attribut `text` zu:

In [5]:
response.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="de" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Python (Programmiersprache) – Wikipedia</title>\n<script>(function(){var className="client-js";var cookie=document.cookie.match(/(?:^|; )dewikimwclientpreferences=([^;]+)/);if(cookie){cookie[1].split(\'%2C\').forEach(function(pref){className=className.replace(new RegExp(\'(^| )\'+pref.replace(/-clientpref-\\w+$|[^\\w-]+/g,\'\')+\'-clientpref-\\\\w+( |$)\'),\'$1\'+pref+\'$2\');});}document.documentElement.className=className;}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.",".\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Januar","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"],"wgRequestId":"7e5c3b53-f988-423f-8d78-773d68c282d2","wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Python_(Programmiersprache)","wgTitle":"Python (P

Dabei erhalten wir einen langen string zurück. Nun wissen wir aber, dass HTML-Code hierarchisch aufgebaut ist. Um die Beziehungen zwischen den einzelnen Elementen zu verdeutlichen, können wir den Quelltext mithilfe des Moduls `BeautifulSoup` sog. *parsen*, also entsprechend der HTML-Syntax interpretieren.

Dazu installieren wir das Modul über die Command Line: `pip3 install beautifulsoup4`. Weiter installieren wir `lxml` (`pip3 install lxml`), das von `BeautifulSoup` intern benötigt wird. Anschließend importieren wir `BeautifulSoup`.

In [6]:
from bs4 import BeautifulSoup #Beachte die korrekte Schreibweise.

Nun können wir den Quelltext parsen, indem wir ein `BeautifulSoup`-Objekt erstellen bzw. eine sog. *Suppe kochen*. Wir übergeben `BeautifulSoup` dazu den Quelltext (und nur diesen, also nicht das gesamte `Response`-Objekt). Anschließend können wir uns die Suppe über die Methode `prettify` schön formatiert ausgeben lassen. 

In [7]:
soup = BeautifulSoup(response.text)
print(soup.prettify()) #'prettify' funktioniert nur innerhalb eines 'print'-Befehls!

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="de">
 <head>
  <meta charset="utf-8"/>
  <title>
   Python (Programmiersprache) – Wikipedia
  </title>
  <script>
   (function(){var className="client-js";var cookie=document.cookie.match(/(?:^|; )dewikimwclientpreferences=([^;]+)/);if(cookie){cookie[1].split('%2C').forEach(function(pref){className=className.replace(new RegExp('(^| )'+pref.replace(/-clientpref-\w+$|[^\w-]+/g,'')+'-clientpref-\\w+( |$)'),'$1'+pref+'$2');});}document.documentElement.className=className;}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Januar","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"],"wgRequestId":"7e5c3b53-f988-423f-8d78-773d68c282d2","wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Python_(Programmiersprache)","wgTitle":"Python (Progra

Zurück zum Abruf von Quelltexten: Die `get`-Funktion von `requests` akzeptiert weitere Parameter, von denen insbesondere zwei wichtig sind: `timeout` und `headers`. 

Über `timeout` definieren wir, wie viele Sekunden `requests` maximal auf eine Antwort vom Server warten soll. Tun wir dies nicht, läuft `requests` im schlimmsten Fall bis in alle Ewigkeit. `headers` wiederum beinhaltet die Informationen, die ein Browser normalerweise an den Server schickt. Indem wir ebensolche Informationen beim Abruf via `requests` verwenden, tun wir so, als würden wir einen gewöhnlichen Browser benutzen. Das minimiert unser Risiko, blockiert zu werden.

In der folgenden Zelle scrapen wir die Startseite der [Tagesschau](https://www.tagesschau.de) mit einem Timeout von fünf Sekunden, nur für den Fall, dass die Webseite gerade down wäre oder wir keinen Zugriff darauf hätten.

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
response = requests.get("https://www.tagesschau.de", timeout=5, headers=headers)

Als Vorblick auf den Extraktionsschritt unten lassen wir uns mal eben die aktuelle, oberste Schlagzeile bei [Tagesschau](https://www.tagesschau.de) ausgeben. Dazu parsen wir den Quelltext wiederum mithilfe von `BeautifulSoup`. Über die `find`-Methode und eine Kombination von Tag (`<span>`) und Attribut (`class="teaser__headline"`) suchen wir nach dem gewünschten Element. 

Dass sich die Schlagzeile in einem `<span>`-Element mit dem Attribut `class="teaser__headline"` befindet, können wir einfach herausfinden, indem wir wie [oben beschrieben](#Hinter-den-Kulissen-von-Webseiten) im Browser das entsprechende Element im Quelltext lokalisieren. 

Sollte der folgende Code eine Fehlermeldung produzieren, liegt das daran, dass der Quellcode der Seite in der Zwischenzeit geändert wurde und ein Element mit der angegebenen Tag-Attribut-Kombination nicht gefunden werden konnte. Bring in diesem Fall selbst in Erfahrung, in welchem Element sich die oberste Schlagzeile befindet. Eine solche wird es bei der Tagesschau ja immer geben.

In [9]:
soup = BeautifulSoup(response.text) #Parsen des eigentlichen Quelltexts
headline = soup.find("span", class_="teaser__headline") #Suchen nach erstem Vorkommen des Elements mit Tag <span> und spezifiertem Attribut (zum Unterstrich in "class_" s. u.)
headline.text #Ausgabe des Textinhalts des Elements

'"Die CDU ist wieder da"'

Wie Du siehst, ist der Abrufschritt sehr trivial. 

Bis hierhin haben wir immer nur eine Webseite auf einmal gescrapt. Mithilfe einer Schleife ist es aber natürlich kein bisschen komplizierter, mehrere Webseiten hintereinander abzurufen. Am einfachsten geht dies, wenn wir bereits eine Liste mit vollständigen Links haben, über die wir iterieren können. Oft können wir uns auch den Umstand zunutze machen, dass URLs von Webseiten nach einem einheitlichen Schema aufgebaut sind. Einen Fall davon wollen wir nun üben.

***

✏️ **Übung 5:** Besuch die Webseite der [Tagesschau](https://www.tagesschau.de) (und überprüf gleich mal, ob die oben extrahierte Schlagzeile wirklich die aktuelle ist!) und ergänz die in `categories` bereits angefangene Liste mit Ressorts, die Nachrichten zu spezifischen Themen bieten. Besuch außerdem die Webseiten einiger Ressorts und analysier, wie die jeweiligen Links aufgebaut sind. Nutz dieses Wissen und iterier über `categories`, um den Quelltext jedes Ressorts herunterzuladen. Benutz denselben Code wie oben, um die oberste Schlagzeile für jedes Ressort zu extrahieren. Lass Dir Ressort und aktuelle oberste Schlagzeile nacheinander ausgeben. 

Schreib den Code für den Abruf- bzw. Extraktionsschritt in separaten Zellen. In der ersten Zelle rufst Du alle Quelltexte ab und hängst sie einer Liste an. In der zweiten Zelle extrahierst Du aus jedem Quelltext die gewünschten Informationen. So führst Du den Abrufschritt nicht mehr aus, wenn Du am Extraktionsschritt arbeitest, wodurch der Server der Tagesschau nicht unnötig belastet wird. Dadurch sinkt das Risiko, dass Du blockiert wirst.

<details><summary>📌 Herausforderung </summary>
<br>Extrahier zusätzlich zur obersten Schlagzeile den Link, der zum entsprechenden Artikel führt und lass ihn Dir ebenfalls ausgeben. Klick auf die ausgegebenen Links, um zu schauen, ob sie wie erwartet funktionieren.
<br><br>
Das entsprechende Element findest Du auf die gleiche Weise wie für die oberste Schlagzeile beschrieben. Da sich der Link aber im <code>href</code>-Attribut (und nicht im Elementinhalt) verbirgt, hängst Du anstatt <code>text</code> die Methode <code>get("href")</code> an das im Quelltext gefundene Element an. 
</details>
<br>


In [10]:
#In diese Zelle kannst Du den Code zum Abrufschritt schreiben.
categories = ["Inland", "Ausland"]




In [11]:
#In diese Zelle kannst Du den Code zum Extraktionsschritt schreiben.




***

In Übung 5 haben wir implizit den Umstand ausgenutzt, dass die Unterseiten von [Tagesschau](https://www.tagesschau.de) analog zur Hauptseite aufgebaut sind. Folglich war es kein Problem, denselben Extraktionscode bei allen Unterseiten anzuwenden. Diese Logik ist matchentscheidend beim Web Scraping! ⚽️

In echten Anwendungsfällen scrapen wir wie hier fast immer mehrere Seiten auf einmal – je nach Forschungsinteresse (Aber-)Tausende. Die eben praktizierte Herangehensweise mit einer `for`-Schleife ist dabei der Standardansatz. Entweder wir verfügen bereits über eine Liste mit kompletten Links oder wir bauen wie eben die Links zu den einzelnen Seiten "on the fly" zusammen.

Eine weitere Variante des iterativen Abrufens von Webseiten kommt bei sog. *paginierten* Seiten zum Einsatz. Paginierte Seiten verteilen zusammengehörenden Inhalt auf mehrere Seiten – ein typisches Beispiel ist ein auf mehreren Seiten fortlaufender Thread in einem Forum (etwa [Stack Overflow](https://stackoverflow.com)). Anstatt zu einer `for`-Schleife, greifen wir in diesem Fall zu einer `while`-Schleife (vgl. Notebook "Kontrollstrukturen"). Auf jeder gescrapten Seite suchen wir nach dem Link zur darauffolgenden Seite, der sich üblicherweise hinter einer Schaltfläche wie "Nächste Seite" verbirgt. Anschließend scrapen wir diese. Die Schleife bricht ab, wenn es keine folgende Seite mehr gibt. Im Anwendungsfall im zweiten Teil dieses Notebooks wirst Du diese Technik einsetzen.

Damit wissen wir alles Wichtige zum Abrufschritt. Machen wir weiter mit der Extraktion!

## Extraktion

Wie oben vorweggenommen parsen wir gescrapte Quelltexte mit `BeautifulSoup`. Das daraus resultierende `BeautifulSoup`-Objekt erlaubt es uns, mit der hierarchischen HTML-Struktur zu arbeiten. 

`BeautifulSoup` hält zahlreiche praktische Techniken bereit, die das Navigieren über HTML-Code sowie das Extrahieren bestimmter Elemente daraus einfach macht. Um diese kennenzulernen, scrapen wir in der folgenden Zelle einen bestimmten [Eintrag](https://scilogs.spektrum.de/hirn-und-weg/schlecht-vernetzt-psychische-stoerungen-im-gehirn/) des Wissenschaftsblogs SciLogs. Es handelt sich um einen Artikel über Hirnforschung im Zusammenhang mit psychischen Erkrankungen. Sollte die Seite in der Zwischenzeit gelöscht worden sein, kannst Du ihren Quelltext stattdessen über die auskommentierten Code-Zeilen einlesen. 

In [12]:
source_code = requests.get("https://scilogs.spektrum.de/hirn-und-weg/schlecht-vernetzt-psychische-stoerungen-im-gehirn/", timeout=5, headers=headers).text

#with open("../3_Dateien/HTML/artikel.html") as f:
    #source_code = f.read()

Als nächstes erstellen wir aus `source_code` ein `BeautifulSoup`-Objekt. 

Als Erinnerung: Wir übergeben `BeautifulSoup` nur den eigentlichen Quelltext, den wir in der Zelle obendran über das `text`-Attribut beim `Response`-Objekt erhalten haben.

In [13]:
soup = BeautifulSoup(source_code) #Falls es Dich interessiert: 'BeautifulSoup' ist ein sog. Constructor bei Python, d. h. es erstellt/initialisiert ein 'BeautifulSoup'-Objekt

Werfen wir einen Blick in den schön formatierten HTML-Code.

In [14]:
print(soup.prettify()) #'prettify' funktioniert wie gesagt nur innerhalb eines 'print'-Befehls!

<!DOCTYPE html>
<html class="no-js" lang="de-DE">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://scilogs.spektrum.de/hirn-und-weg/xmlrpc.php" rel="pingback"/>
  <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
  <title>
   Schlecht vernetzt? Psychische Störungen im Gehirn » HIRN UND WEG » SciLogs - Wissenschaftsblogs
  </title>
  <meta content="Welche Theorien haben wir dazu wie psychische Störungen im Gehirn aussehen und wie können wir dies erforschen? Neuronale Netzwerke" name="description"/>
  <link href="https://scilogs.spektrum.de/hirn-und-weg/schlecht-vernetzt-psychische-stoerungen-im-gehirn/" rel="canonical"/>
  <meta content="de_DE" property="og:locale"/>
  <meta content="article" property="og:type"/>
  <meta content="Schlecht vernetzt? Psychische Störungen im Gehirn" property="og:title

Scroll einmal durch den Quelltext durch. Öffne ebenfalls die [Webseite](https://scilogs.spektrum.de/hirn-und-weg/schlecht-vernetzt-psychische-stoerungen-im-gehirn/) und lokalisier einige wichtige Elemente wie Überschriften, Bildunterschriften oder die Kommentare im Quelltext. Bei der weiteren Arbeit mit diesem Quelltext – sowie wenn Du später eigene Webseiten scrapst – kommst Du nicht umhin, Dich immer wieder in den Quelltext "reinzufuchsen".

Wie oben gesehen, können wir mithilfe der Methode `find`, das erste Element des als Argument übergebenen Tags aus dem Quelltext extrahieren. Hier tun wir dies für den ersten Textparagraphen:

In [15]:
soup.find("p")

<p>Die Neurowissenschaften haben ein großes übergeordnetes Ziel: Zu verstehen, wie unser Nervensystem funktioniert. Die zentralste Rolle spielt dabei unser Gehirn: Schon seit Jahrhunderten sind Menschen fasziniert von dem Organ, versuchen unser Denken und Handeln zu verstehen. Von besonderem Interesse ist dabei auch immer die Frage, was passiert, wenn Funktionen im Gehirn gestört sind bzw. welche gestörten Funktionen verschiedenen psychischen Störungen zugrunde liegen könnten.</p>

Die Methode gibt uns das ganze Element inkl. Start- und Endtag (sowie, falls vorhanden, Attribute) zurück. Um den Textinhalt zu isolieren, können wir wie oben bereits demonstriert das Attribut `text` anhängen:

In [16]:
first_paragraph = soup.find("p") #Das Ergebnis der Suche lässt sich natürlich auch zwischenzeitlich in einer Variablen speichern.
first_paragraph.text #Sehr komfortabel: 'text' ist ein Attribut sowohl bei 'Response'- als auch bei 'BeautifulSoup'-Objekten

'Die Neurowissenschaften haben ein großes übergeordnetes Ziel: Zu verstehen, wie unser Nervensystem funktioniert. Die zentralste Rolle spielt dabei unser Gehirn: Schon seit Jahrhunderten sind Menschen fasziniert von dem Organ, versuchen unser Denken und Handeln zu verstehen. Von besonderem Interesse ist dabei auch immer die Frage, was passiert, wenn Funktionen im Gehirn gestört sind bzw. welche gestörten Funktionen verschiedenen psychischen Störungen zugrunde liegen könnten.'

`BeautifulSoup` ermöglicht es einem auch, über die dot-Notation (vgl. Notebook "Datenanalyse Teil 1") auf das erste Element eines gewünschten Tags zuzugreifen:

In [17]:
soup.p
#soup.p.text #Zugriff nur auf den Textinhalt

<p>Die Neurowissenschaften haben ein großes übergeordnetes Ziel: Zu verstehen, wie unser Nervensystem funktioniert. Die zentralste Rolle spielt dabei unser Gehirn: Schon seit Jahrhunderten sind Menschen fasziniert von dem Organ, versuchen unser Denken und Handeln zu verstehen. Von besonderem Interesse ist dabei auch immer die Frage, was passiert, wenn Funktionen im Gehirn gestört sind bzw. welche gestörten Funktionen verschiedenen psychischen Störungen zugrunde liegen könnten.</p>

Wie wir oben auch schon gesehen haben, können wir neben dem gesuchten Tag auch weitere Suchkriterien angeben, etwa ein bestimmtes Schlüssel-Werte-Paar bei den Attributen. Unter den Textparagraphen im gescrapten Artikel gibt es welche, die einer bestimmten `class` zugeordnet sind, etwa der Abschnitt über die Autorin des Artikels. Dieser verfügt über das Attribut `class="author-description"`. Angenommen wir sind an genau dieser Information interessiert, können wir zusätzlich zum Tag `<p>` nach dem entsprechenden Attribut suchen.

⚠️ Achtung: `class` ist ein "reserviertes" Schlüsselwort bei Python, es entspricht dem, was wir unter Datentypen verstehen (vgl. Notebooks "Einführung" und "Datentypen"). Um einen Konflikt mit Python zu verhindern, hängt man ganz einfach einen Unterstrich an: `class_`.

In [18]:
soup.find("p", class_="author-description").text #Ausgabe nur des Textinhalts

'Mein Name ist Lea Anthes und ich studiere Klinische Psychologie und Psychotherapie im Master an der Goethe-Universität in Frankfurt. Ich interessiere mich schon lange für Themen rund um das menschliche Gehirn und konnte mich während meines Bachelorstudiums der Psychologie sowohl umfangreich mit der kognitiven Neurowissenschaft auseinandersetzen als auch praktische Erfahrung im Bereich der klinischen Neuropsychologie sammeln. Gerne teile ich diese Begeisterung mit interessierten Leserinnen und Lesern.'

Dadurch erhalten wir nicht mehr das erste beliebige `<p>`-Element im Quelltext, sondern das erste, das außerdem über das gesuchte Attribut verfügt.

***

✏️ **Übung 6:** Extrahier die oberste Überschrift ("Schlecht vernetzt? Psychische Störungen im Gehirn") sowie die erste Überschrift auf der nächstniedrigeren Ebene ("Hirnforschung: Damals und heute") aus dem Quelltext. Extrahier ebenfalls die erste Bildunterschrift ("Die Phrenologie (Bildquelle)"). 

<details>
  <summary>💡 Tipp</summary>
  <br>Als Erstes musst Du dafür herausfinden, mit welchen Tags diese Elemente versehen sind. 
</details>
<br>

In [19]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Wie oben gesehen, erhalten wir durch `find` stets das ganze Element zurück. Dieses Element kann sich irgendwo in der Hierarchie des HTML-Stammbaums befinden. Entsprechend können ihm auch (viele) weitere Elemente untergeordnet sein. Was wir auf `soup` im Großen anwenden können (etwa die Methode `find`), können wir auch auf ein durch `find` zurückgegebenes Element im Kleinen anwenden.

Machen wir's konkret: Der in Übung 6 extrahierten ersten Bildunterschrift ist ein weiteres Element untergeordnet. Überleg Dir kurz, worum es sich dabei handelt und schau ggf. auf der Webseite nach. 

Genau, es handelt sich um das (Inline-)Element `<a>`, also um einen Link. Dieses Element extrahieren wir ganz einfach mit derselben Logik wie bis anhin, d.&nbsp;h. wir wenden `find` ein zweites Mal auf das durch `soup.find("figcaption")` zurückgegebene Element an.

In [20]:
#Drei Wege zum Ziel
print(soup.find("figcaption").find("a")) #Zweimal mithilfe von 'find'
print(soup.figcaption.a) #Zweimal mit dot-Notation
print(soup.find("figcaption").a) #'find' und dot-Notation lassen sich auch kombinieren (auch umgekehrt)

<a href="https://pixabay.com/de/illustrations/jahrgang-gehirn-werbung-idee-1418613/">Bildquelle</a>
<a href="https://pixabay.com/de/illustrations/jahrgang-gehirn-werbung-idee-1418613/">Bildquelle</a>
<a href="https://pixabay.com/de/illustrations/jahrgang-gehirn-werbung-idee-1418613/">Bildquelle</a>


Wiederum erhalten wir das ganze Element. 

Angenommen wir sind nur am Link interessiert, so können wir nicht die bisher verwendete Technik, das `text`-Attribut, einsetzen, um darauf zuzugreifen. Schließlich befindet sich der Link im Wert des `href`-Attributs und nicht im Textinhalt (da steht "Bildquelle"). 

Stattdessen verwenden wir eine Methode namens `get` und übergeben ihr als Argument das gewünschte Attribut (bzw. den Schlüssel):

In [21]:
soup.figcaption.a.get("href") #'get' erinnert Dich vielleicht an dictionaries, Attribute von HTML-Elementen sind auch wie dictionaries aufgebaut.

'https://pixabay.com/de/illustrations/jahrgang-gehirn-werbung-idee-1418613/'

Klappt wunderbar!

Was, wenn wir nicht bloß am ersten Treffer eines bestimmten Suchkriteriums interessiert sind, sondern an allen? Dann kommt die Methode `find_all` ins Spiel. Da die Methode eine Liste mit den gefundenen Elementen zurückgibt, bauen wir sie am besten gleich in eine Schleife ein. 

Im folgenden, kompakten Code erstellen wir zunächst eine Liste mit allen `<a>`-Elementen, also allen Links (`soup.find_all("a")`), iterieren darüber (`for link in ...`) und greifen bei jedem `link` auf den Wert seines `href`-Attributs zu (`link.get("href")`).

In [22]:
#Beachte den Unterschied in der Schreibweise im Vergleich zur 'findall'-Funktion für reguläre Ausdrücke
for link in soup.find_all("a"):
    print(link.get("href")) 
    
#Verkürzte Syntax für 'find_all', die 'soup' wie eine Funktion behandelt
#for link in soup("a"):
    #print(link.get("href"))

https://www.spektrum.de
/
https://scilogs.spektrum.de/network-tax/category/astronomie
https://scilogs.spektrum.de/astrogeo
https://scilogs.spektrum.de/go-for-launch
https://scilogs.spektrum.de/relativ-einfach
https://scilogs.spektrum.de/sternbildung
https://scilogs.spektrum.de/uhura-uraniae
https://scilogs.spektrum.de/network-tax/category/biologie
https://scilogs.spektrum.de/beobachtungen-der-wissenschaft
https://scilogs.spektrum.de/die-sankore-schriften
https://scilogs.spektrum.de/hier-wohnen-drachen
https://scilogs.spektrum.de/marlenes-medizinkiste
https://scilogs.spektrum.de/meertext
https://scilogs.spektrum.de/menschen-bilder
https://scilogs.spektrum.de/via-data
https://scilogs.spektrum.de/network-tax/category/chemie
https://scilogs.spektrum.de/fischblog
https://scilogs.spektrum.de/via-data
https://scilogs.spektrum.de/network-tax/category/erde-umwelt
https://scilogs.spektrum.de/astrogeo
https://scilogs.spektrum.de/der-anthropozaeniker
https://scilogs.spektrum.de/fischblog
https://s

Et voilà: alle Links aus dem Quelltext dieser Seite. 

Angenommen wir sind nur an den Links aus dem eigentlichen Artikel interessiert, lohnt es sich, statt des kompletten Quelltexts (`soup`) nur den relevanten Teil davon abzusuchen. Dazu müssen wir im Quelltext herausfinden, welches am niedrigsten in der HTML-Hierarchie befindliche Element trotzdem den gesamten Artikelinhalt "unter sich" enthält. Zwar gibt es ein Element mit Tag `<article>`, das den Quelltext schon sinnvoll "zuschneidet", folgende Tag-Attribut-Kombination ist aber noch etwas spezifischer:

In [23]:
actual_blog_post = soup.find("div", class_="hentry__content post__content hentry__content--is-content post__content--is-content")

Mit `actual_blog_post` haben wir ein Element, das den relevanten Teil des Quelltexts enthält. Dieses können wir nun wie gewohnt nach Links absuchen:

In [24]:
for link in actual_blog_post.find_all("a"):
    print(link.get("href"))

https://scilogs.spektrum.de/hirn-und-weg/files/Phrenologie.png
https://pixabay.com/de/illustrations/jahrgang-gehirn-werbung-idee-1418613/
https://scilogs.spektrum.de/hirn-und-weg/ki-uebersetzt-hirnsignale-in-sprache/
https://scilogs.spektrum.de/hirn-und-weg/files/MRT.jpg
https://de.freepik.com/fotos-kostenlos/maennlicher-radiologe-analysiert-mri-scan-ergebnisse-eines-patienten-auf-einem-computermonitor-im-kontrollraum_26143902.htm#query=fmri&position=7&from_view=search&track=ais&uuid=4a7bca36-3a27-4110-a6d7-f90ab4cf52dc
https://scilogs.spektrum.de/hirn-und-weg/leben-mit-adhs/
https://scilogs.spektrum.de/hirn-und-weg/files/Default_mode_network.jpg
https://commons.wikimedia.org/wiki/File:Default_mode_network-WRNMMC.jpg
https://scilogs.spektrum.de/hirn-und-weg/was-tut-unser-hirn-wenn-wir-nichts-tun/
https://commons.wikimedia.org/wiki/File:Fnbeh-08-00171-g002.jpg
https://www.spektrum.de/magazin/konnektomik-netzwerke-im-gehirn/1734134
https://doi.org/10.1016/j.tics.2011.08.003
https://www.d

Damit wären wir unserem Ziel schon wesentlich näher. Nach demselben Prinzip könnten wir bei Bedarf noch spezifischer extrahieren, etwa um die Links zum Teilen des Artikels (via Facebook, Xing, etc.) herauszufiltern. 

In vielen Fällen macht es Sinn, zunächst über `find` ein `BeautifulSoup`-Objekt zu erstellen, das möglichst wenig des gesamten Quelltexts umfasst, aber alles davon, was wir anschließend extrahieren wollen. Es handelt sich dabei um eine Art Vorsortierung, bevor wir uns an die eigentliche Extraktion machen.

Genau wie `find` können wir auch `find_all` mehr als ein Suchkriterium übergeben. Stellen wir uns vor, wir sind an allen Überschriften *innerhalb* des Artikels interessiert. Wie wir in Übung 6 herausgefunden haben, sind diese Teil eines `<h3>`-Elements. Folgender Code mit nur dem Tag als Suchauftrag ist noch zu unspezifisch, liefert er uns doch auch entsprechende Elemente von außerhalb des Artikels (sog. *False Positives*, vgl. "Die Wahrheitsmatrix" in Notebook "Input und Output Teil 1"):

In [25]:
soup.find_all("h3")
#soup("h3") #Verkürzte Syntax

[<h3 class="wp-block-heading" id="h-hirnforschung-damals-und-heute">Hirnforschung: Damals und heute</h3>,
 <h3 class="wp-block-heading" id="h-was-sind-neurokognitive-large-scale-brain-networks">Was sind neurokognitive „large-scale brain networks“?</h3>,
 <h3 class="wp-block-heading" id="h-wie-neuronale-netze-gemessen-werden">Wie neuronale Netze gemessen werden</h3>,
 <h3 class="wp-block-heading" id="h-drei-grosse-netzwerke">Drei große Netzwerke</h3>,
 <h3 class="wp-block-heading" id="h-ein-neuronales-netzwerkmodell-psychischer-storungen">Ein neuronales Netzwerkmodell psychischer Störungen</h3>,
 <h3 class="wp-block-heading" id="h-fazit">Fazit</h3>,
 <h3 class="wp-block-heading" id="h-quellen">Quellen</h3>,
 <h3>Veröffentlicht von <a href="https://scilogs.spektrum.de/hirn-und-weg/author/anthes/" rel="author" title="Lea Anthes">Lea Anthes</a></h3>,
 <h3 id="comments-title"> 7 Kommentare </h3>,
 <h3 class="comment-reply-title" id="reply-title">Schreibe einen Kommentar <small><a href="/hir

Ein kurzer Blick in die Ausgabe verrät aber, dass alle gewünschten Überschriften (und nur diese) über das Attribut `class="wp-block-heading"` spezifiziert sind. Bauen wir dieses in den Suchauftrag ein:

In [26]:
soup.find_all("h3", class_="wp-block-heading")
#soup("h3", class_="wp-block-heading") #Verkürzte Syntax

[<h3 class="wp-block-heading" id="h-hirnforschung-damals-und-heute">Hirnforschung: Damals und heute</h3>,
 <h3 class="wp-block-heading" id="h-was-sind-neurokognitive-large-scale-brain-networks">Was sind neurokognitive „large-scale brain networks“?</h3>,
 <h3 class="wp-block-heading" id="h-wie-neuronale-netze-gemessen-werden">Wie neuronale Netze gemessen werden</h3>,
 <h3 class="wp-block-heading" id="h-drei-grosse-netzwerke">Drei große Netzwerke</h3>,
 <h3 class="wp-block-heading" id="h-ein-neuronales-netzwerkmodell-psychischer-storungen">Ein neuronales Netzwerkmodell psychischer Störungen</h3>,
 <h3 class="wp-block-heading" id="h-fazit">Fazit</h3>,
 <h3 class="wp-block-heading" id="h-quellen">Quellen</h3>]

Das klappt hervorragend!

Wie hätten wir diese sieben Überschriften noch extrahieren können? 

Genau, indem wir anstatt `soup` das maßgeschneiderte Element `actual_blog_post` abgesucht hätten, wodurch das Suchkriterium des Attributs obsolet geworden wäre:

In [27]:
#Überprüfung, ob die beiden Herangehensweisen dasselbe Resultat zeitigen
soup.find_all("h3", class_="wp-block-heading") == actual_blog_post.find_all("h3")

True

Neben Tag-Attribut-Kombinationen können wir unsere Suchkriterien bei `find_all` weiter verfeinern und u.&nbsp;a. folgendes übergeben:

- eine Liste mit Tags statt einem einzelnen Tag
- ein regulärer Ausdruck für Tags bzw. Werte bei Attributen (bzw. Schlüsseln; vgl. Notebook "Reguläre Ausdrücke")
- eine Liste mit Werten für ein bestimmtes Attribut (bzw. Schlüssel) statt einem einzigen Wert

In der folgenden Zelle werden diese Suchkriterien exemplifiziert:

In [28]:
#Liste mit Tags statt einem einzelnen Tag
for header in soup.find_all(["h1", "h2", "h3"]):
    print(header.text.strip()) #Bereinigung von leading/trailing whitespace
    
print("\n")

#Regulärer Ausdruck für Tags (beachte: 'find_all' erwartet einen mittels 're.compile' kompilierten regulären Ausdruck).
import re

for header in soup.find_all(re.compile("h[1-3]")):
    print(header.text.strip()) #Bereinigung von leading/trailing whitespace

print("\n")

#Liste mit Werten für 'class'-Attribut
for element in soup.find_all(class_=["newsletter-area", "widgets-after-content"]):
    print(element.text.strip(), "\n") #Bereinigung von leading/trailing whitespace

Schlecht vernetzt? Psychische Störungen im Gehirn
Hirnforschung: Damals und heute
Was sind neurokognitive „large-scale brain networks“?
Wie neuronale Netze gemessen werden
Drei große Netzwerke
Ein neuronales Netzwerkmodell psychischer Störungen
Fazit
Quellen
Veröffentlicht von Lea Anthes
7 Kommentare
Schreibe einen Kommentar Antworten abbrechen


Schlecht vernetzt? Psychische Störungen im Gehirn
Hirnforschung: Damals und heute
Was sind neurokognitive „large-scale brain networks“?
Wie neuronale Netze gemessen werden
Drei große Netzwerke
Ein neuronales Netzwerkmodell psychischer Störungen
Fazit
Quellen
Veröffentlicht von Lea Anthes
7 Kommentare
Schreibe einen Kommentar Antworten abbrechen


HIRN UND WEG ist der Neuroblog der Gemeinnützigen Hertie-Stiftung , der die Bandbreite und Facetten eines der faszinierendsten Organe zeigen, Erkenntnisse aus Wissenschaft einfach und gut erklären und geistreich und unterhaltsam begeistern möchte. Neben der Informationsvermittlung gehören die Förderun

Lass uns das bisher Erlernte zum Extraktionsschritt in die Tat umsetzen!

***

✏️ **Übung 7:** In dieser Übung sollst Du die Kommentare unter dem Artikel extrahieren. Identifizier dazu ein oder mehrere Suchkriterien im Quelltext, anhand derer Du alle Kommentare extrahieren kannst. Konzentrier Dich vorerst auf die Kommentare auf der ersten Ebene (ignorier also Kommentare zu Kommentaren). 

Lass Dir für jeden Kommentar folgende Informationen ausgeben:

1. Wer hat den Kommentar verfasst?
2. Wann wurde der Kommentar verfasst?
3. Wie lautet der Text des Kommentars?

<details>
  <summary>💡 Tipp 1</summary>
  <br>Extrahier zunächst ein <code>BeautifulSoup</code>-Objekt, das möglichst wenig vom gesamten Quelltext umfasst, aber genug, um alle Kommentare zu enthalten. Arbeite anschließend damit. 
</details>
<br>
<details>
  <summary>💡 Tipp 2</summary>
  <br>Es gibt verschiedene Kombinationen von Suchkriterien, die Dich ans Ziel bringen. Eine verwendet das <code>class</code>-Attribut, das bei den gewünschten Elementen in mehreren, leicht unterschiedlichen Ausführungen vorkommt. Wie kannst Du mehrere Werte für ein Attribut spezifizieren?
</details>
<br>
<details>
  <summary>💡 Tipp 3</summary>
    <br>Wendest Du das <code>text</code>-Attribut auf ein Element mit untergeordneten Elementen an, so erhältst Du sämtlichen Textinhalt, auch denjenigen der Unterelemente, konkateniert zurück. Setz diese Tatsache produktiv ein!
</details>
<br>
<details><summary>📌 Herausforderung </summary>
<br>Extrahier zusätzlich die Kommentare zu den Kommentaren und lass sie Dir unterhalb des zugehörigen Hauptkommentars, wiederum mit Verfasser:in, Datum und Text, ausgeben. Benutz dafür einen regulären Ausdruck.
</details>
<br>

In [29]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Oben haben wir diverse Schlagzeilen von unterschiedlichen Tagesschau-Ressortseiten mit ein und demselben Code extrahiert. Das hat nur deshalb geklappt, weil die verschiedenen Seiten einheitlich aufgebaut sind. In der nächsten Übung wollen wir sehen, ob dieser Umstand auch für das Wissenschaftsblog SciLogs gilt (Spoiler: Natürlich tut es das). 

***

✏️ **Übung 8:** Begib Dich auf die [SciLogs-Startseite](https://scilogs.spektrum.de) und trag einige Links zu Blogeinträgen manuell in `links` zusammen. Ruf in derselben Zelle die Quelltexte zu allen Seiten ab und speichere sie in einer weiteren Liste. 

Parse anschließend in der zweiten Zelle iterativ einen Quelltext nach dem anderen und extrahier jeweils die Kommentare. Hierfür kannst Du grundsätzlich den gleichen Code wie in Übung 7 verwenden. Pass ihn aber (sofern nötig) so an, dass nun alle Kommentare extrahiert werden, auch jene auf zweiter, dritter etc. Ebene. Stell sicher, dass Du auch wirklich alle Kommentare pro Blogeintrag extrahierst.

Lass Dir zusätzlich zu den Informationen aus Übung 7 vor den Kommentaren jeweils den Titel des Blogeintrags ausgeben.

<details>
  <summary>💡 Tipp</summary>
  <br>Verwend wieder das <code>class</code>-Attribut, um nach den Kommentaren zu suchen. Wie schon in Übung 7 bemerkt, gibt es mehrere mögliche Werte, die Du in den Suchfilter einbauen musst. Da wir jetzt außerdem Kommentare auf zweiter, dritter etc. Ebene extrahieren, nimmt die Zahl an möglichen Werten nochmal zu. Zwar kannst Du nach wie vor mit einer Liste an möglichen Werten arbeiten, doch drängt sich ein regulärer Ausdruck stark auf. Solltest Du mit regulären Ausdrücken noch nicht vertraut sein, dann kopier den entsprechenden Code aus der Lösung zur Herausforderung bei Übung 7. 
</details>
<br>

In [30]:
#Abrufschritt
links = []



In [31]:
#Extraktionsschritt




*** 

Sehr gut!

Nun haben wir gelernt, wie wir *zielgerichtet* relevante Informationen aus langen Quelltexten extrahieren können.

Wie versprochen lernen wir zum Abschluss des ersten Teils dieses Notebooks noch einen unkomplizierten Weg kennen, der sich dann anbietet, wenn wir ganz *generell* am Haupttext einer Webseite interessiert sind. Wir arbeiten dazu mit dem Modul `trafilatura`.

## Texte sammeln mit `trafilatura`

`trafilatura` ist mit den gängigsten HTML-Strukturen vertraut und weiß, in welchen Elementen sich üblicherweise dieser Haupttext verbirgt. Das Modul trumpft insbesondere bei großen Datensammlungen, in denen der (gesamte) Haupttext von Webseiten im Fokus steht, z.&nbsp;B. wenn wir für ein Korpus sämtliche Texte aller Artikel von ZEIT Online aus einem bestimmten Zeitraum sammeln wollen. 

`trafilatura` lässt sich unabhängig von `requests` und `BeautifulSoup` verwenden, da es sowohl den Abruf- als auch den Extraktionsschritt beherrscht.

Installiere `trafilatura` erst über die Command Line (`pip3 install trafilatura`) und setz es anschließend wie folgt ein:

In [32]:
import trafilatura
downloaded = trafilatura.fetch_url('https://scilogs.spektrum.de/hirn-und-weg/schlecht-vernetzt-psychische-stoerungen-im-gehirn/') #Abrufschritt
trafilatura.extract(downloaded) #Extraktionsschritt

ModuleNotFoundError: No module named 'trafilatura'

Wenn Du durch die Ausgabe scrollst, siehst Du, dass `trafilatura` den Artikeltext (inkl. Überschriften) sowie die Kommentare extrahiert hat – vollkommen ohne einen Blick unsererseits in den Quelltext. 

`extract` akzeptiert weitere Parameter, über die sich der Extraktionsschritt verfeinern lässt, z.&nbsp;B. kann über `output_format` das gewünschte Extraktionsformat spezifiziert werden (u.&nbsp;a. csv oder XML, vgl. zweiter Teil dieses Notebooks). 

***

**✏️ Übung 9:** Benutz `trafilatura`, um Dir die Artikeltexte sämtlicher Blogeinträge ausgeben zu lassen, deren URLs Du in Übung 8 in `links` zusammengetragen hast. Recherchier in der [Dokumentation](https://trafilatura.readthedocs.io/en/latest/corefunctions.html#trafilatura.bare_extraction) von `trafilatura`, wie Du die Kommentare dabei herausfiltern kannst. Lass Dir bei jedem Blogeintrag den Link, die ersten 300 Zeichen des Texts, das Auslassungszeichen [...] sowie die finalen 300 Zeichen ausgeben.

In [33]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Damit sind wir am Ende des ersten Teils dieses Notebooks angelangt. Gute Arbeit bis hierhin! 🎉

Weiter geht's im zweiten Teil des Notebooks "Web Scraping".

<br><br>
***
<table>
      <tr>
        <td>
            <img src="../3_Dateien/Lizenz/CC-BY-SA.png" width="400">
        </td> 
        <td>
            <p>Dieses Notebook sowie sämtliche weiteren <a href="https://github.com/yannickfrommherz/exdimed-student/tree/main">Materialien zum Programmierenlernen für Geistes- und Sozialwissenschaftler:innen</a> sind im Rahmen des Projekts <i>Experimentierraum Digitale Medienkompetenz</i> als Teil von <a href="https://tu-dresden.de/gsw/virtuos/">virTUos</a> entstanden. Erstellt wurden sie von Yannick Frommherz unter Mitarbeit von Anne Josephine Matz. Sie stehen als Open Educational Resource nach <a href="https://creativecommons.org/licenses/by-sa/4.0/">CC BY SA</a> zur freien Verfügung. Für Feedback und bei Fragen nutz bitte das <a href="https://forms.gle/VsYJgy4bZTSqKioA7">Kontaktformular</a>.
        </td>
      </tr>
</table>